In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import soundfile
# root_path = "F:/DATAS/covid-19-main/dataset-main/"
# metafilename = "metadata.json"
df_f = pd.read_pickle("F:/DATAS/covid-19-main/dataset-main/covid19_split_balancevalid_df.pkl")
#df_f = pd.read_json(root_path + metafilename)

# 随机地抽取出100正100负样本

In [ ]:
import random
neg_list = list(range(100)) + list(range(200, 1109))
pos_list = list(range(100, 200)) + list(range(1109, 2733))
random.shuffle(neg_list)
random.shuffle(pos_list)
valid_list = neg_list[:100] + pos_list[:100]
train_list = neg_list[100:] + pos_list[100:]
train_df = self.us8k_df.iloc[train_list, :]
valid_df = self.us8k_df.iloc[valid_list, :]

# The metadata.json file contains a description of samples using the following fields:
1. filename - name of the file which is in the directory raw2. 
start_at - the beginning of an induced cough, se
3. 
end_at - end of induced cough, s
4. c
verified - record verified (optional fie
5. d)
asymptomatic - asymptomatic (optional fi
6. ld)
comment - co
7. ment
covid19 - feature of existing C  
8. VID19
source - source of seedf seed

In [ ]:

df_f = pd.read_json(root_path + metafilename)
# df_f["asymptomatic"] = df_f["asymptomatic"].fillna(2)
df_f

In [ ]:
# df_f = df_f.dropna(subset=["verified"])
print("--> shape:", df_f.shape)  # 1324 7
print(df_f.groupby("verified")["filename"].count())  # sum to 1120
print(df_f.groupby("asymptomatic")["filename"].count())  # sum to 674
print(df_f.groupby("covid19")["filename"].count())  # sum to 1324
print(df_f.groupby("source")["filename"].count())  # sum to 1324
print(df_f.groupby("kind")["filename"].count())  # sum to 204

# 计算每个音频的时长然后保存csv

In [ ]:
dura_list = []

# for idx, row in tqdm(enumerate(df_f.iterrows()), total=len(df_f)):
for idx, row in tqdm(enumerate(df_f.itertuples()), total=len(df_f)):
    # print(idx, row)
    fname = root_path + "raw/" +getattr(row, "filename")
    try:
        samples, sample_rate = soundfile.read(fname, dtype='float32')
        duration = samples.shape[0] / float(sample_rate)
        print(sample_rate, end=', ')
        dura_list.append(duration)
    except Exception as e:
        print(e)
        print(fname)
        dura_list.append(99999)
print(dura_list)

In [ ]:
len(dura_list)

In [ ]:
df_f["dura"] = dura_list
df_f.to_csv("F:/DATAS/covid-19-main/dataset-main/covid19-cough_metadata.csv", sep=',')

# 切分音频然后保存csv

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import soundfile
root_path = "F:/DATAS/covid-19-main/dataset-main/"
metafilename = "metadata.json"
df_f = pd.read_json(root_path + metafilename)
m2l = {True:0, False:1}
df_f["covid19"] = df_f["covid19"].apply(lambda x:m2l[x])
print(df_f)
print(df_f.groupby("covid19")["filename"].count())  # sum to 1324

In [ ]:
import librosa
def segment_cough(x,fs, cough_padding=0.2,min_cough_len=0.2, th_l_multiplier = 0.1, th_h_multiplier = 2):
    #Preprocess the data by segmenting each file into individual coughs using a hysteresis comparator on the signal power                
    cough_mask = np.array([False]*len(x))
    
    #Define hysteresis thresholds
    rms = np.sqrt(np.mean(np.square(x)))
    seg_th_l = th_l_multiplier * rms
    seg_th_h =  th_h_multiplier*rms

    #Segment coughs
    coughSegments = []
    padding = round(fs*cough_padding)
    min_cough_samples = round(fs*min_cough_len)
    cough_start = 0
    cough_end = 0
    cough_in_progress = False
    tolerance = round(0.01*fs)
    below_th_counter = 0
    
    for i, sample in enumerate(x**2):
        if cough_in_progress:
            if sample<seg_th_l:
                below_th_counter += 1
                if below_th_counter > tolerance:
                    cough_end = i+padding if (i+padding < len(x)) else len(x)-1
                    cough_in_progress = False
                    if (cough_end+1-cough_start-2*padding>min_cough_samples):
                        coughSegments.append(x[cough_start:cough_end+1])
                        cough_mask[cough_start:cough_end+1] = True
            elif i == (len(x)-1):
                cough_end=i
                cough_in_progress = False
                if (cough_end+1-cough_start-2*padding>min_cough_samples):
                    coughSegments.append(x[cough_start:cough_end+1])
            else:
                below_th_counter = 0
        else:
            if sample>seg_th_h:
                cough_start = i-padding if (i-padding >=0) else 0
                cough_in_progress = True
    
    return coughSegments, cough_mask

In [ ]:
HOP_LENGTH = 512        # number of samples between successive frames
WINDOW_LENGTH = 512     # length of the window in samples
N_MEL = 128             # number of Mel bands to generate


def compute_melspectrogram_with_fixed_length(audio, sampling_rate, num_of_samples=128):
    try:
        # compute a mel-scaled spectrogram
        melspectrogram = librosa.feature.melspectrogram(y=audio, 
                                                        sr=sampling_rate, 
                                                        hop_length=HOP_LENGTH,
                                                        win_length=WINDOW_LENGTH, 
                                                        n_mels=N_MEL)

        # convert a power spectrogram to decibel units (log-mel spectrogram)
        melspectrogram_db = librosa.power_to_db(melspectrogram, ref=np.max)
        
        melspectrogram_length = melspectrogram_db.shape[1]
        
        # # pad or fix the length of spectrogram 
        # if melspectrogram_length != num_of_samples:
        #     melspectrogram_db = librosa.util.fix_length(melspectrogram_db, 
        #                                                 size=num_of_samples, 
        #                                                 axis=1, 
        #                                                 constant_values=(0, -80.0))
    except Exception as e:
        print("\nError encountered while parsing files\n>>", e)
        return None 
    
    return melspectrogram_db

## 切分咳嗽数据并复制标签

In [ ]:
audio_length = 32306
# sample_rate = 22050
all_data = []
all_fname = []
all_sr = []
all_labels = []
new_df = df_f
maxi, mini = 0, 999999
for idx, row in tqdm(enumerate(df_f.itertuples()), total=len(df_f)):
    fname = root_path + "raw/" +getattr(row, "filename")
    label = getattr(row, "covid19")
    # load sound sample
    try:
        audio, sample_rate = librosa.load(fname, mono=True)
    except Exception as e:
        # print(e)
        print("Error file:", fname)
        continue
    maxi = max(maxi, audio.shape[0]/sample_rate)
    mini = min(mini, audio.shape[0]/sample_rate)
    # Segment each audio into individual coughs using a hysteresis comparator on the signal power
    cough_segments, cough_mask = segment_cough(audio, sample_rate, min_cough_len=0.1, cough_padding=0.1, th_l_multiplier = 0.1, th_h_multiplier = 2)

    # For each segment, resize to the same length(11025)
    if len(cough_segments) > 0 :
        i = 0
        for audio in cough_segments:
            i+=1
            if len(audio) > 8000:
                if len(audio) < audio_length:
                    audio_pad = librosa.util.pad_center(data=audio, size=audio_length)
                else:
                    audio_pad = audio[:audio_length]
                # audio_pad = audio

            # feature = extract_features(audio_pad, sample_rate)
            #print(len(feature))
            # all_data.append(feature)
            all_data.append(audio_pad)
            all_fname.append(fname)
            all_labels.append(label)
            all_sr.append(sample_rate)
            new_df = pd.concat([new_df, df_f.iloc[[idx], :]], axis=0)
new_df = new_df.iloc[len(df_f):, :]

In [ ]:
print(maxi, mini)
print(len(new_df))
print(len(all_data))
print(max([item.shape[0] for item in all_data]))

# fold标签的生成
- 这里不再随机了，而是根据label的0 和 1 分为两份，然后按照数目较少的一组进行双指针分配。
- 这么做不行：设置一个计数器，保证每个fold里面的正负标签数目一致，然后最终选0号fold作为测试集。因为政府并不均衡。

In [ ]:
# melspects = []
# import random
features_1 = []
features_2 = []
# 
for i in tqdm(range(len(all_data)),desc="calc.."):
    melspect = compute_melspectrogram_with_fixed_length(all_data[i], all_sr[i])
    # print(melspec.shape)
    # melspects.append(melspect)
    # print(neg_idx, pos_idx)
    if all_labels[i] == 0:
        features_1.append([melspect, 0])
    else:
        features_2.append([melspect, 1])

In [ ]:
print(len(features_1), len(features_2))
# 2733 

In [ ]:
features = []
fold1 = []
fold2 = []
for i in range(len(features_1)):
    if i < 100:
        fold1.append([features_1[i][0], features_1[i][1], 0])
    else:
        fold2.append([features_1[i][0], features_1[i][1], random.randint(1, 9)])
        
import random
for i in range(len(features_2)):
    if i < 100:
        fold1.append([features_2[i][0], features_2[i][1], 0])
    else:
        fold2.append([features_2[i][0], features_2[i][1], random.randint(1, 9)])
for j, item in enumerate(fold1):
    print(j, item[0].shape, item[1], item[2])
for j, item in enumerate(fold2):
    print(j, item[0].shape, item[1], item[2])

In [ ]:
features = []
features.extend(fold1)
features.extend(fold2)

In [ ]:
us8k_df = pd.DataFrame(features, columns=["melspectrogram", "label", "fold"])
us8k_df.to_pickle(root_path+"covid19_split_balancevalid_df.pkl")

In [ ]:
train_df = us8k_df[us8k_df['fold'] != 0]
valid_df = us8k_df[us8k_df['fold'] == 0]

In [ ]:
valid_df

In [ ]:
new_df["filename"] = new_fnames
new_df.to_csv(root_path+"covid19_split_df.csv", sep=',')

# test pkl

In [ ]:
import pandas as pd
df = pd.read_pickle("F:/DATAS/covid-19-main/dataset-main/covid19_split_df.pkl")

In [ ]:
print(df.groupby("label")["melspectrogram"].count())

In [ ]:
for item in df.itertuples():
    print(getattr(item, "melspectrogram").shape)

# End

In [ ]:
import numpy as np
mini, maxi = np.floor(min(dura_list)), np.ceil(max(dura_list))
print(mini, maxi)
L = maxi - mini
cnt = [0] * int(L)
for item in dura_list:
    cnt[int(item)] += 1
print(cnt)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(cnt, bins=30, color='skyblue', alpha=0.8)

# 设置图表属性
plt.title('RUNOOB hist() Test')
plt.xlabel('Value')
plt.ylabel('Frequency')

# 显示图表
plt.show()

In [ ]:
duras = []
for item in all_data:
    print(item.shape)